In [ ]:
!pip install tsplib95

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 17.4 MB/s eta 0:00:00
  Attempting uninstall: tabulate
    Found existing installation: tabulate 0.9.0
    Uninstalling tabulate-0.9.0:
      Successfully uninstalled tabulate-0.9.0
  Attempting uninstall: networkx
    Found existing installation: networkx 3.3
    Uninstalling networkx-3.3:
      Successfully uninstalled networkx-3.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 1.21.0 requires tabulate>=0.9, but you have tabulate 0.8.10 which is incompatible.


In [ ]:
# Run 1: population_size=100, max_gen=6, elite_count=2, mutation_prob=0.01
import numpy as np
import tsplib95
import random
import math
import time
from collections import namedtuple

# Load TSPLIB95 dataset
def load_tsplib95_instance(filename):
    problem = tsplib95.load(filename)
    if problem.is_depictable():
        nodes = np.array([problem.get_display(node) for node in problem.get_nodes()])
    else:
        print("This problem instance does not have display data.")
        nodes = None
    return nodes

# Calculate the distance matrix
def calculate_distance_matrix(nodes):
    n = len(nodes)
    dist_matrix = np.zeros((n, n))
    for i in range(n):
        for j in range(n):
            dist_matrix[i, j] = np.linalg.norm(nodes[i] - nodes[j])
    return dist_matrix

# Define the BBO algorithm
class BBO:
    def __init__(self, dist_matrix, population_size=100, max_gen=6, elite_count=2, mutation_prob=0.01):
        self.dist_matrix = dist_matrix
        self.population_size = population_size
        self.max_gen = max_gen
        self.elite_count = elite_count
        self.mutation_prob = mutation_prob
        self.population = []
        self.elite = []
        self.best_solution = None
        self.best_cost = float('inf')

    def initialize_population(self):
        for _ in range(self.population_size):
            individual = np.random.permutation(len(self.dist_matrix))
            cost = self.calculate_cost(individual)
            self.population.append((individual, cost))

    def calculate_cost(self, individual):
        cost = 0
        for i in range(1, len(individual)):
           cost += self.dist_matrix[individual[i - 1], individual[i]]
        # Adding the cost to return to the starting point to complete the cycle
        cost += self.dist_matrix[individual[-1], individual[0]]
        return cost

    def calculate_latency(self, individual):
        latency = 0
        total_latency = 0
        for i in range(1, len(individual)):
            latency += self.dist_matrix[individual[i - 1], individual[i]]
            total_latency += latency
        return total_latency

    def migrate(self, emigrate, immigrate):
        for i in range(len(immigrate)):
            if random.random() < self.mutation_prob:
                swap_idx = np.random.randint(0, len(emigrate))
                immigrate[i], immigrate[swap_idx] = immigrate[swap_idx], immigrate[i]
        return immigrate

    def optimize(self):
        start_time = time.time()
        self.initialize_population()
        self.population.sort(key=lambda x: x[1])

        for gen in range(self.max_gen):
            new_population = self.population[:self.elite_count]
            for i in range(self.elite_count, self.population_size):
                emigrate_idx = random.randint(0, self.elite_count - 1)
                immigrate_idx = random.randint(self.elite_count, self.population_size - 1)
                emigrate = self.population[emigrate_idx][0].copy()
                immigrate = self.population[immigrate_idx][0].copy()
                new_individual = self.migrate(emigrate, immigrate)
                new_cost = self.calculate_latency(new_individual)
                new_population.append((new_individual, new_cost))

            self.population = new_population
            self.population.sort(key=lambda x: x[1])

            if self.population[0][1] < self.best_cost:
                self.best_cost = self.population[0][1]
                self.best_solution = self.population[0][0]
            print(f"Generation {gen+1}: Best Cost = {self.best_cost}")

        end_time = time.time()
        avg_time = (end_time - start_time) / self.max_gen

        return self.best_solution, self.best_cost * 10, avg_time * 10

# Load the dataset
filename = r'/content/drive/MyDrive/TSPLIB95 folder/rat195.tsp'
nodes = load_tsplib95_instance(filename)
dist_matrix = calculate_distance_matrix(nodes)

# Run BBO algorithm
bbo = BBO(dist_matrix, population_size=100, max_gen=6, elite_count=2, mutation_prob=0.01)
best_solution, best_cost, avg_time = bbo.optimize()

print("Best solution found:", best_solution)
print("Cost of the best solution:", best_cost)
print("Average time per generation:", avg_time)

Generation 1: Best Cost = 20791.428926830395
Generation 2: Best Cost = 20791.428926830395
Generation 3: Best Cost = 20791.428926830395
Generation 4: Best Cost = 20791.428926830395
Generation 5: Best Cost = 20791.428926830395
Generation 6: Best Cost = 20791.428926830395
Best solution found: [181 161 178 155 152  27  51 136 175  14 130 184 183  84  65 140 127  10
 165 171  32  87  17 151  75  34   3  50 115 189 113  59  21  19  52  47
  95 147  72   1  69 139  56 144  66 107 128  26  55  73 120 146   4  77
  12 143 104 105  13  24 169 159  96 174 168  62   6 134  64 138  42 182
 106  58  60  85  92  82  40  28  91  23  74 170 150  57 188  90  43  71
  48 116 187 109  18 160 121 125 102  76 185 100  61   9 112  68  35 154
 137  88 103  38  86  78 180 132  37 158 191 135 118 111  49 192 119  30
  36  89 142 156 179  25  54  16  98 164 177 172 167   5 145 194  67 162
 131  39   8  63  45 148  97  31  44 114  22 157  70 124 186 190 193 149
 153  99 163 173 108 110 122  81  11  53 117 176  79

In [ ]:
# Run 2: population_size=100, max_gen=6, elite_count=3, mutation_prob=0.02
import numpy as np
import tsplib95
import random
import math
import time
from collections import namedtuple

# Load TSPLIB95 dataset
def load_tsplib95_instance(filename):
    problem = tsplib95.load(filename)
    if problem.is_depictable():
        nodes = np.array([problem.get_display(node) for node in problem.get_nodes()])
    else:
        print("This problem instance does not have display data.")
        nodes = None
    return nodes

# Calculate the distance matrix
def calculate_distance_matrix(nodes):
    n = len(nodes)
    dist_matrix = np.zeros((n, n))
    for i in range(n):
        for j in range(n):
            dist_matrix[i, j] = np.linalg.norm(nodes[i] - nodes[j])
    return dist_matrix

# Define the BBO algorithm
class BBO:
    def __init__(self, dist_matrix, population_size=100, max_gen=6, elite_count=3, mutation_prob=0.02):
        self.dist_matrix = dist_matrix
        self.population_size = population_size
        self.max_gen = max_gen
        self.elite_count = elite_count
        self.mutation_prob = mutation_prob
        self.population = []
        self.elite = []
        self.best_solution = None
        self.best_cost = float('inf')

    def initialize_population(self):
        for _ in range(self.population_size):
            individual = np.random.permutation(len(self.dist_matrix))
            cost = self.calculate_cost(individual)
            self.population.append((individual, cost))

    def calculate_cost(self, individual):
        cost = 0
        for i in range(1, len(individual)):
           cost += self.dist_matrix[individual[i - 1], individual[i]]
        # Adding the cost to return to the starting point to complete the cycle
        cost += self.dist_matrix[individual[-1], individual[0]]
        return cost

    def calculate_latency(self, individual):
        latency = 0
        total_latency = 0
        for i in range(1, len(individual)):
            latency += self.dist_matrix[individual[i - 1], individual[i]]
            total_latency += latency
        return total_latency

    def migrate(self, emigrate, immigrate):
        for i in range(len(immigrate)):
            if random.random() < self.mutation_prob:
                swap_idx = np.random.randint(0, len(emigrate))
                immigrate[i], immigrate[swap_idx] = immigrate[swap_idx], immigrate[i]
        return immigrate

    def optimize(self):
        start_time = time.time()
        self.initialize_population()
        self.population.sort(key=lambda x: x[1])

        for gen in range(self.max_gen):
            new_population = self.population[:self.elite_count]
            for i in range(self.elite_count, self.population_size):
                emigrate_idx = random.randint(0, self.elite_count - 1)
                immigrate_idx = random.randint(self.elite_count, self.population_size - 1)
                emigrate = self.population[emigrate_idx][0].copy()
                immigrate = self.population[immigrate_idx][0].copy()
                new_individual = self.migrate(emigrate, immigrate)
                new_cost = self.calculate_latency(new_individual)
                new_population.append((new_individual, new_cost))

            self.population = new_population
            self.population.sort(key=lambda x: x[1])

            if self.population[0][1] < self.best_cost:
                self.best_cost = self.population[0][1]
                self.best_solution = self.population[0][0]
            print(f"Generation {gen+1}: Best Cost = {self.best_cost}")

        end_time = time.time()
        avg_time = (end_time - start_time) / self.max_gen

        return self.best_solution, self.best_cost * 10, avg_time * 10

# Load the dataset
filename = r'/content/drive/MyDrive/TSPLIB95 folder/rat195.tsp'
nodes = load_tsplib95_instance(filename)
dist_matrix = calculate_distance_matrix(nodes)

# Run BBO algorithm
bbo = BBO(dist_matrix, population_size=100, max_gen=6, elite_count=3, mutation_prob=0.02)
best_solution, best_cost, avg_time = bbo.optimize()

print("Best solution found:", best_solution)
print("Cost of the best solution:", best_cost)
print("Average time per generation:", avg_time)

Generation 1: Best Cost = 20626.364839601545
Generation 2: Best Cost = 20626.364839601545
Generation 3: Best Cost = 20626.364839601545
Generation 4: Best Cost = 20626.364839601545
Generation 5: Best Cost = 20626.364839601545
Generation 6: Best Cost = 20626.364839601545
Best solution found: [ 73 150 166 189 176 126 133 129 138 187 141 163 144 182 181   5  41   1
  34  14 193  11  31  45  90   2   7 151  77 102 180  25 192  59  61 179
  95 112 159  51 111 109  97  68 183 137  29 152  56  98  99  38  33  86
 116 190  87  17  82  47 175 164 118 165 157 162  50 161 188 171 106  94
 140  24 143  22 146  84  66   0  30  62  37 105  58  44 120 185 139 142
  36  27  55  57  74   8  32  18 167 135  53  75  20  35  64  52 125 121
  91  28   3 177 124  16 131 128 186 174 178  72  49 194 184 127  48 122
   4 114 170  71   9  88  96  42 154 153  93 147  63  67 119  83  40 191
  19  78 123 155  70 158 149 110  92   6 173 113  69 104 103  13  15 115
 172 117  26 134  80 130 168 136 107 132 101  89  54

In [ ]:
# Run 3: population_size=450, max_gen=6, elite_count=22, mutation_prob=0.045
import numpy as np
import tsplib95
import random
import math
import time
from collections import namedtuple

# Load TSPLIB95 dataset
def load_tsplib95_instance(filename):
    problem = tsplib95.load(filename)
    if problem.is_depictable():
        nodes = np.array([problem.get_display(node) for node in problem.get_nodes()])
    else:
        print("This problem instance does not have display data.")
        nodes = None
    return nodes

# Calculate the distance matrix
def calculate_distance_matrix(nodes):
    n = len(nodes)
    dist_matrix = np.zeros((n, n))
    for i in range(n):
        for j in range(n):
            dist_matrix[i, j] = np.linalg.norm(nodes[i] - nodes[j])
    return dist_matrix

# Define the BBO algorithm
class BBO:
    def __init__(self, dist_matrix, population_size=450, max_gen=6, elite_count=22, mutation_prob=0.045):
        self.dist_matrix = dist_matrix
        self.population_size = population_size
        self.max_gen = max_gen
        self.elite_count = elite_count
        self.mutation_prob = mutation_prob
        self.population = []
        self.elite = []
        self.best_solution = None
        self.best_cost = float('inf')

    def initialize_population(self):
        for _ in range(self.population_size):
            individual = np.random.permutation(len(self.dist_matrix))
            cost = self.calculate_cost(individual)
            self.population.append((individual, cost))

    def calculate_cost(self, individual):
        cost = 0
        for i in range(1, len(individual)):
           cost += self.dist_matrix[individual[i - 1], individual[i]]
        # Adding the cost to return to the starting point to complete the cycle
        cost += self.dist_matrix[individual[-1], individual[0]]
        return cost

    def calculate_latency(self, individual):
        latency = 0
        total_latency = 0
        for i in range(1, len(individual)):
            latency += self.dist_matrix[individual[i - 1], individual[i]]
            total_latency += latency
        return total_latency

    def migrate(self, emigrate, immigrate):
        for i in range(len(immigrate)):
            if random.random() < self.mutation_prob:
                swap_idx = np.random.randint(0, len(emigrate))
                immigrate[i], immigrate[swap_idx] = immigrate[swap_idx], immigrate[i]
        return immigrate

    def optimize(self):
        start_time = time.time()
        self.initialize_population()
        self.population.sort(key=lambda x: x[1])

        for gen in range(self.max_gen):
            new_population = self.population[:self.elite_count]
            for i in range(self.elite_count, self.population_size):
                emigrate_idx = random.randint(0, self.elite_count - 1)
                immigrate_idx = random.randint(self.elite_count, self.population_size - 1)
                emigrate = self.population[emigrate_idx][0].copy()
                immigrate = self.population[immigrate_idx][0].copy()
                new_individual = self.migrate(emigrate, immigrate)
                new_cost = self.calculate_latency(new_individual)
                new_population.append((new_individual, new_cost))

            self.population = new_population
            self.population.sort(key=lambda x: x[1])

            if self.population[0][1] < self.best_cost:
                self.best_cost = self.population[0][1]
                self.best_solution = self.population[0][0]
            print(f"Generation {gen+1}: Best Cost = {self.best_cost}")

        end_time = time.time()
        avg_time = (end_time - start_time) / self.max_gen

        return self.best_solution, self.best_cost * 10, avg_time * 10

# Load the dataset
filename = r'/content/drive/MyDrive/TSPLIB95 folder/rat195.tsp'
nodes = load_tsplib95_instance(filename)
dist_matrix = calculate_distance_matrix(nodes)

# Run BBO algorithm
bbo = BBO(dist_matrix, population_size=450, max_gen=6, elite_count=22, mutation_prob=0.045)
best_solution, best_cost, avg_time = bbo.optimize()

print("Best solution found:", best_solution)
print("Cost of the best solution:", best_cost)
print("Average time per generation:", avg_time)

Generation 1: Best Cost = 19898.39016168285
Generation 2: Best Cost = 19898.39016168285
Generation 3: Best Cost = 19898.39016168285
Generation 4: Best Cost = 19898.39016168285
Generation 5: Best Cost = 19898.39016168285
Generation 6: Best Cost = 19898.39016168285
Best solution found: [ 82  31  81 117  43 157 176 191  61  22  13  87 111  90  64 152 140 127
  37  80 182  52 119  68 139 132 129  86  70  34  19 190  25  75 121  26
  23  47 118  30  57  46   7 188  60 108  62 116  84 122  18  97   6 161
 185   5  14  16 143 149 194  50  33 156 134 164  39 172 115 154 147 137
  63 107 155 165 128 168  24  45  95  83  65  11  48  49 103  96  78  94
  92  74 153 181 167 166   4 104 130 177  69 145 178 189 142  20  29 125
  77  89   8  76 105  58  40 174 133 150  93 179  12  72 106  67  53  17
 146  54 123 151 109  44  36 113  85  59  21 162 114 102 158 193 148 138
 131  51  27   0  38 136  15   9 184 169  55 101 110 163 171  42  32   2
  10 175 173 112 160  91  71 126  88  79  41 187   1   3  

In [ ]:
# Run 4: population_size=350, max_gen=6, elite_count=17, mutation_prob=0.035
import numpy as np
import tsplib95
import random
import math
import time
from collections import namedtuple

# Load TSPLIB95 dataset
def load_tsplib95_instance(filename):
    problem = tsplib95.load(filename)
    if problem.is_depictable():
        nodes = np.array([problem.get_display(node) for node in problem.get_nodes()])
    else:
        print("This problem instance does not have display data.")
        nodes = None
    return nodes

# Calculate the distance matrix
def calculate_distance_matrix(nodes):
    n = len(nodes)
    dist_matrix = np.zeros((n, n))
    for i in range(n):
        for j in range(n):
            dist_matrix[i, j] = np.linalg.norm(nodes[i] - nodes[j])
    return dist_matrix

# Define the BBO algorithm
class BBO:
    def __init__(self, dist_matrix, population_size=350, max_gen=6, elite_count=17, mutation_prob=0.035):
        self.dist_matrix = dist_matrix
        self.population_size = population_size
        self.max_gen = max_gen
        self.elite_count = elite_count
        self.mutation_prob = mutation_prob
        self.population = []
        self.elite = []
        self.best_solution = None
        self.best_cost = float('inf')

    def initialize_population(self):
        for _ in range(self.population_size):
            individual = np.random.permutation(len(self.dist_matrix))
            cost = self.calculate_cost(individual)
            self.population.append((individual, cost))

    def calculate_cost(self, individual):
        cost = 0
        for i in range(1, len(individual)):
           cost += self.dist_matrix[individual[i - 1], individual[i]]
        # Adding the cost to return to the starting point to complete the cycle
        cost += self.dist_matrix[individual[-1], individual[0]]
        return cost

    def calculate_latency(self, individual):
        latency = 0
        total_latency = 0
        for i in range(1, len(individual)):
            latency += self.dist_matrix[individual[i - 1], individual[i]]
            total_latency += latency
        return total_latency

    def migrate(self, emigrate, immigrate):
        for i in range(len(immigrate)):
            if random.random() < self.mutation_prob:
                swap_idx = np.random.randint(0, len(emigrate))
                immigrate[i], immigrate[swap_idx] = immigrate[swap_idx], immigrate[i]
        return immigrate

    def optimize(self):
        start_time = time.time()
        self.initialize_population()
        self.population.sort(key=lambda x: x[1])

        for gen in range(self.max_gen):
            new_population = self.population[:self.elite_count]
            for i in range(self.elite_count, self.population_size):
                emigrate_idx = random.randint(0, self.elite_count - 1)
                immigrate_idx = random.randint(self.elite_count, self.population_size - 1)
                emigrate = self.population[emigrate_idx][0].copy()
                immigrate = self.population[immigrate_idx][0].copy()
                new_individual = self.migrate(emigrate, immigrate)
                new_cost = self.calculate_latency(new_individual)
                new_population.append((new_individual, new_cost))

            self.population = new_population
            self.population.sort(key=lambda x: x[1])

            if self.population[0][1] < self.best_cost:
                self.best_cost = self.population[0][1]
                self.best_solution = self.population[0][0]
            print(f"Generation {gen+1}: Best Cost = {self.best_cost}")

        end_time = time.time()
        avg_time = (end_time - start_time) / self.max_gen

        return self.best_solution, self.best_cost * 10, avg_time * 10

# Load the dataset
filename = r'/content/drive/MyDrive/TSPLIB95 folder/rat195.tsp'
nodes = load_tsplib95_instance(filename)
dist_matrix = calculate_distance_matrix(nodes)

# Run BBO algorithm
bbo = BBO(dist_matrix, population_size=350, max_gen=6, elite_count=17, mutation_prob=0.035)
best_solution, best_cost, avg_time = bbo.optimize()

print("Best solution found:", best_solution)
print("Cost of the best solution:", best_cost)
print("Average time per generation:", avg_time)

Generation 1: Best Cost = 20460.235798599264
Generation 2: Best Cost = 20460.235798599264
Generation 3: Best Cost = 20460.235798599264
Generation 4: Best Cost = 20460.235798599264
Generation 5: Best Cost = 20460.235798599264
Generation 6: Best Cost = 20460.235798599264
Best solution found: [156  34 157  79 101  67 137 108 106 132 166  49 113 165 180  27  33 158
  47  28 121  58 109 190 182 192 187  50  20  23  78 178 144 122 179 170
  30  90  24  95 146 186  55 169 155 147 134 129  86  31  26  10 107  92
  29 116 124 130  32  62 141  74  81  22 168 128  76  80  70 161 102 119
 105   1   2  43  25  66  77   8 149 193 162  88  53  36 160  61  45   9
 185  72  89  38 139 126 103  75 114  42 125  82 127 159  65  19 111  15
  48   6  14  18 154  99  87  51 117 135 152 104 153 167 191  39 177  84
 171  12  73 115  97 176 163  69  11 131 142  54  46  35  17  21  83 181
  37  93 188 133 136  16 123 145 140 151 194   4  56 184 183 189  13  68
  57  59  52   7 173  98 100 120 175  64  60  63   3

In [ ]:
# Run 5: population_size=250, max_gen=6, elite_count=12, mutation_prob=0.025
import numpy as np
import tsplib95
import random
import math
import time
from collections import namedtuple

# Load TSPLIB95 dataset
def load_tsplib95_instance(filename):
    problem = tsplib95.load(filename)
    if problem.is_depictable():
        nodes = np.array([problem.get_display(node) for node in problem.get_nodes()])
    else:
        print("This problem instance does not have display data.")
        nodes = None
    return nodes

# Calculate the distance matrix
def calculate_distance_matrix(nodes):
    n = len(nodes)
    dist_matrix = np.zeros((n, n))
    for i in range(n):
        for j in range(n):
            dist_matrix[i, j] = np.linalg.norm(nodes[i] - nodes[j])
    return dist_matrix

# Define the BBO algorithm
class BBO:
    def __init__(self, dist_matrix, population_size=250, max_gen=6, elite_count=12, mutation_prob=0.025):
        self.dist_matrix = dist_matrix
        self.population_size = population_size
        self.max_gen = max_gen
        self.elite_count = elite_count
        self.mutation_prob = mutation_prob
        self.population = []
        self.elite = []
        self.best_solution = None
        self.best_cost = float('inf')

    def initialize_population(self):
        for _ in range(self.population_size):
            individual = np.random.permutation(len(self.dist_matrix))
            cost = self.calculate_cost(individual)
            self.population.append((individual, cost))

    def calculate_cost(self, individual):
        cost = 0
        for i in range(1, len(individual)):
           cost += self.dist_matrix[individual[i - 1], individual[i]]
        # Adding the cost to return to the starting point to complete the cycle
        cost += self.dist_matrix[individual[-1], individual[0]]
        return cost

    def calculate_latency(self, individual):
        latency = 0
        total_latency = 0
        for i in range(1, len(individual)):
            latency += self.dist_matrix[individual[i - 1], individual[i]]
            total_latency += latency
        return total_latency

    def migrate(self, emigrate, immigrate):
        for i in range(len(immigrate)):
            if random.random() < self.mutation_prob:
                swap_idx = np.random.randint(0, len(emigrate))
                immigrate[i], immigrate[swap_idx] = immigrate[swap_idx], immigrate[i]
        return immigrate

    def optimize(self):
        start_time = time.time()
        self.initialize_population()
        self.population.sort(key=lambda x: x[1])

        for gen in range(self.max_gen):
            new_population = self.population[:self.elite_count]
            for i in range(self.elite_count, self.population_size):
                emigrate_idx = random.randint(0, self.elite_count - 1)
                immigrate_idx = random.randint(self.elite_count, self.population_size - 1)
                emigrate = self.population[emigrate_idx][0].copy()
                immigrate = self.population[immigrate_idx][0].copy()
                new_individual = self.migrate(emigrate, immigrate)
                new_cost = self.calculate_latency(new_individual)
                new_population.append((new_individual, new_cost))

            self.population = new_population
            self.population.sort(key=lambda x: x[1])

            if self.population[0][1] < self.best_cost:
                self.best_cost = self.population[0][1]
                self.best_solution = self.population[0][0]
            print(f"Generation {gen+1}: Best Cost = {self.best_cost}")

        end_time = time.time()
        avg_time = (end_time - start_time) / self.max_gen

        return self.best_solution, self.best_cost * 10, avg_time * 10

# Load the dataset
filename = r'/content/drive/MyDrive/TSPLIB95 folder/rat195.tsp'
nodes = load_tsplib95_instance(filename)
dist_matrix = calculate_distance_matrix(nodes)

# Run BBO algorithm
bbo = BBO(dist_matrix, population_size=250, max_gen=6, elite_count=12, mutation_prob=0.025)
best_solution, best_cost, avg_time = bbo.optimize()

print("Best solution found:", best_solution)
print("Cost of the best solution:", best_cost)
print("Average time per generation:", avg_time)

Generation 1: Best Cost = 20517.187934373625
Generation 2: Best Cost = 20517.187934373625
Generation 3: Best Cost = 20517.187934373625
Generation 4: Best Cost = 20517.187934373625
Generation 5: Best Cost = 20517.187934373625
Generation 6: Best Cost = 20517.187934373625
Best solution found: [130  21  84  34  96  66 162 183  90  60  54  91 123  97  18  53 172 157
 166 143 176 114 104  80  95 101 145 188  39 148  63  55  62 102  98 105
 179 163 192  50  24  67 127  38 184 161 106 120 137 169  86 135   0  36
   9  82  23  78  83 103  29  57  32  25  59 150 170 194 171  45  75 111
 134 151  15  17  46  65 128  61  81 187  72 159 140 167 190 117  42 193
 168 181  28 125   7 164   6 155   4  85 141  47  19  33  12 139 100 119
 153 133 186  93 152 173  30   1 189 191  68 185  26  64 132 122  52   3
 110   8  35  74 178  76 174  71 124 154  69  22 144 160 108 146  27  87
 142 158 116   5  37  89  56  11 129  77  48 118 112 182   2  14 177 136
 147 107 121  31  88  49 175 165 138 113 156  43  20

In [ ]:
# Run 6: population_size=150, max_gen=6, elite_count=30, mutation_prob=0.06
import numpy as np
import tsplib95
import random
import math
import time
from collections import namedtuple

# Load TSPLIB95 dataset
def load_tsplib95_instance(filename):
    problem = tsplib95.load(filename)
    if problem.is_depictable():
        nodes = np.array([problem.get_display(node) for node in problem.get_nodes()])
    else:
        print("This problem instance does not have display data.")
        nodes = None
    return nodes

# Calculate the distance matrix
def calculate_distance_matrix(nodes):
    n = len(nodes)
    dist_matrix = np.zeros((n, n))
    for i in range(n):
        for j in range(n):
            dist_matrix[i, j] = np.linalg.norm(nodes[i] - nodes[j])
    return dist_matrix

# Define the BBO algorithm
class BBO:
    def __init__(self, dist_matrix, population_size=150, max_gen=6, elite_count=30, mutation_prob=0.06):
        self.dist_matrix = dist_matrix
        self.population_size = population_size
        self.max_gen = max_gen
        self.elite_count = elite_count
        self.mutation_prob = mutation_prob
        self.population = []
        self.elite = []
        self.best_solution = None
        self.best_cost = float('inf')

    def initialize_population(self):
        for _ in range(self.population_size):
            individual = np.random.permutation(len(self.dist_matrix))
            cost = self.calculate_cost(individual)
            self.population.append((individual, cost))

    def calculate_cost(self, individual):
        cost = 0
        for i in range(1, len(individual)):
           cost += self.dist_matrix[individual[i - 1], individual[i]]
        # Adding the cost to return to the starting point to complete the cycle
        cost += self.dist_matrix[individual[-1], individual[0]]
        return cost

    def calculate_latency(self, individual):
        latency = 0
        total_latency = 0
        for i in range(1, len(individual)):
            latency += self.dist_matrix[individual[i - 1], individual[i]]
            total_latency += latency
        return total_latency

    def migrate(self, emigrate, immigrate):
        for i in range(len(immigrate)):
            if random.random() < self.mutation_prob:
                swap_idx = np.random.randint(0, len(emigrate))
                immigrate[i], immigrate[swap_idx] = immigrate[swap_idx], immigrate[i]
        return immigrate

    def optimize(self):
        start_time = time.time()
        self.initialize_population()
        self.population.sort(key=lambda x: x[1])

        for gen in range(self.max_gen):
            new_population = self.population[:self.elite_count]
            for i in range(self.elite_count, self.population_size):
                emigrate_idx = random.randint(0, self.elite_count - 1)
                immigrate_idx = random.randint(self.elite_count, self.population_size - 1)
                emigrate = self.population[emigrate_idx][0].copy()
                immigrate = self.population[immigrate_idx][0].copy()
                new_individual = self.migrate(emigrate, immigrate)
                new_cost = self.calculate_latency(new_individual)
                new_population.append((new_individual, new_cost))

            self.population = new_population
            self.population.sort(key=lambda x: x[1])

            if self.population[0][1] < self.best_cost:
                self.best_cost = self.population[0][1]
                self.best_solution = self.population[0][0]
            print(f"Generation {gen+1}: Best Cost = {self.best_cost}")

        end_time = time.time()
        avg_time = (end_time - start_time) / self.max_gen

        return self.best_solution, self.best_cost * 10, avg_time * 10

# Load the dataset
filename = r'/content/drive/MyDrive/TSPLIB95 folder/rat195.tsp'
nodes = load_tsplib95_instance(filename)
dist_matrix = calculate_distance_matrix(nodes)

# Run BBO algorithm
bbo = BBO(dist_matrix, population_size=150, max_gen=6, elite_count=30, mutation_prob=0.06)
best_solution, best_cost, avg_time = bbo.optimize()

print("Best solution found:", best_solution)
print("Cost of the best solution:", best_cost)
print("Average time per generation:", avg_time)

Generation 1: Best Cost = 21066.418878334065
Generation 2: Best Cost = 21066.418878334065
Generation 3: Best Cost = 21066.418878334065
Generation 4: Best Cost = 21066.418878334065
Generation 5: Best Cost = 21066.418878334065
Generation 6: Best Cost = 21066.418878334065
Best solution found: [ 31  52   9 146 173 186 141 154 102 125  51  30  74  97 135 189 163 111
 178 175  29  32  94 113   4 142  82   7  33  66   6  49 158  90 101 148
 136 118  18  79  39 143 112 183 119 153  36 188 179 122  59  43 147 192
  37  71 172  53  56 184 193 165  68   5  84   2  23 167  92 191 108   0
  10  99 149  91   8  73  85  25 152 157  27 107  83 187  40  57  95 150
  69  24  22 160  11  76 132 145 120  72  67 182  55  20 159 161  42  80
  98  47 129  14  41 126 106 194 124  87  70 133 121  96  54 128  26 168
 140  63 105  61 131 156  50  19 155 151 162 117  13  62  38 190  15 130
  46   1 110 138 123 169 181 114 180 185 115 139 164  35  88 137  44  60
  34  16  17  12 176 134  45 170  86  75  93  28 104

In [ ]:
# Run 7: population_size=500, max_gen=6, elite_count=15, mutation_prob=0.01
import numpy as np
import tsplib95
import random
import math
import time
from collections import namedtuple

# Load TSPLIB95 dataset
def load_tsplib95_instance(filename):
    problem = tsplib95.load(filename)
    if problem.is_depictable():
        nodes = np.array([problem.get_display(node) for node in problem.get_nodes()])
    else:
        print("This problem instance does not have display data.")
        nodes = None
    return nodes

# Calculate the distance matrix
def calculate_distance_matrix(nodes):
    n = len(nodes)
    dist_matrix = np.zeros((n, n))
    for i in range(n):
        for j in range(n):
            dist_matrix[i, j] = np.linalg.norm(nodes[i] - nodes[j])
    return dist_matrix

# Define the BBO algorithm
class BBO:
    def __init__(self, dist_matrix, population_size=500, max_gen=6, elite_count=15, mutation_prob=0.01):
        self.dist_matrix = dist_matrix
        self.population_size = population_size
        self.max_gen = max_gen
        self.elite_count = elite_count
        self.mutation_prob = mutation_prob
        self.population = []
        self.elite = []
        self.best_solution = None
        self.best_cost = float('inf')

    def initialize_population(self):
        for _ in range(self.population_size):
            individual = np.random.permutation(len(self.dist_matrix))
            cost = self.calculate_cost(individual)
            self.population.append((individual, cost))

    def calculate_cost(self, individual):
        cost = 0
        for i in range(1, len(individual)):
           cost += self.dist_matrix[individual[i - 1], individual[i]]
        # Adding the cost to return to the starting point to complete the cycle
        cost += self.dist_matrix[individual[-1], individual[0]]
        return cost

    def calculate_latency(self, individual):
        latency = 0
        total_latency = 0
        for i in range(1, len(individual)):
            latency += self.dist_matrix[individual[i - 1], individual[i]]
            total_latency += latency
        return total_latency

    def migrate(self, emigrate, immigrate):
        for i in range(len(immigrate)):
            if random.random() < self.mutation_prob:
                swap_idx = np.random.randint(0, len(emigrate))
                immigrate[i], immigrate[swap_idx] = immigrate[swap_idx], immigrate[i]
        return immigrate

    def optimize(self):
        start_time = time.time()
        self.initialize_population()
        self.population.sort(key=lambda x: x[1])

        for gen in range(self.max_gen):
            new_population = self.population[:self.elite_count]
            for i in range(self.elite_count, self.population_size):
                emigrate_idx = random.randint(0, self.elite_count - 1)
                immigrate_idx = random.randint(self.elite_count, self.population_size - 1)
                emigrate = self.population[emigrate_idx][0].copy()
                immigrate = self.population[immigrate_idx][0].copy()
                new_individual = self.migrate(emigrate, immigrate)
                new_cost = self.calculate_latency(new_individual)
                new_population.append((new_individual, new_cost))

            self.population = new_population
            self.population.sort(key=lambda x: x[1])

            if self.population[0][1] < self.best_cost:
                self.best_cost = self.population[0][1]
                self.best_solution = self.population[0][0]
            print(f"Generation {gen+1}: Best Cost = {self.best_cost}")

        end_time = time.time()
        avg_time = (end_time - start_time) / self.max_gen

        return self.best_solution, self.best_cost * 10, avg_time * 10

# Load the dataset
filename = r'/content/drive/MyDrive/TSPLIB95 folder/rat195.tsp'
nodes = load_tsplib95_instance(filename)
dist_matrix = calculate_distance_matrix(nodes)

# Run BBO algorithm
bbo = BBO(dist_matrix, population_size=500, max_gen=6, elite_count=15, mutation_prob=0.01)
best_solution, best_cost, avg_time = bbo.optimize()

print("Best solution found:", best_solution)
print("Cost of the best solution:", best_cost)
print("Average time per generation:", avg_time)

Generation 1: Best Cost = 20937.088991108998
Generation 2: Best Cost = 20937.088991108998
Generation 3: Best Cost = 20937.088991108998
Generation 4: Best Cost = 20937.088991108998
Generation 5: Best Cost = 20937.088991108998
Generation 6: Best Cost = 20937.088991108998
Best solution found: [ 31  54 126 117   5 145  11  44  97  70  78  72  76  82  21  32  25  49
  37  86  99 191 163 165  85 169 151  16  33  36 189 160 133  29  57   4
 110 105 180  26  69  45  64 187 183  81 137  38  68 155 192 129 184   3
 176 121 141 128 125 120 142  28 182 149 112   7 168 156  91 130   9  61
 190 107 135 164 158 162  12  19  55  34   2  18 132 172 103  94 188 136
 174  52 143  80 185 131 177 146  15  17 101 193  59 181  93  63  46 102
  58  90 152 157 134 115  13 140  62   1  79 139  47 123  75  23 116  96
  53  98  84 161  66 100 173  74 150 113  83 138  95 148 194 122  35  14
  71  88  56 124  42 171 119 111 106  92   6  10  40  51   0  50  87 159
  89 127  67 186 108  39   8 170  24 154  43  22  77

In [ ]:
# Run 8: population_size=300, max_gen=6, elite_count=20, mutation_prob=1
import numpy as np
import tsplib95
import random
import math
import time
from collections import namedtuple

# Load TSPLIB95 dataset
def load_tsplib95_instance(filename):
    problem = tsplib95.load(filename)
    if problem.is_depictable():
        nodes = np.array([problem.get_display(node) for node in problem.get_nodes()])
    else:
        print("This problem instance does not have display data.")
        nodes = None
    return nodes

# Calculate the distance matrix
def calculate_distance_matrix(nodes):
    n = len(nodes)
    dist_matrix = np.zeros((n, n))
    for i in range(n):
        for j in range(n):
            dist_matrix[i, j] = np.linalg.norm(nodes[i] - nodes[j])
    return dist_matrix

# Define the BBO algorithm
class BBO:
    def __init__(self, dist_matrix, population_size=300, max_gen=6, elite_count=20, mutation_prob=1):
        self.dist_matrix = dist_matrix
        self.population_size = population_size
        self.max_gen = max_gen
        self.elite_count = elite_count
        self.mutation_prob = mutation_prob
        self.population = []
        self.elite = []
        self.best_solution = None
        self.best_cost = float('inf')

    def initialize_population(self):
        for _ in range(self.population_size):
            individual = np.random.permutation(len(self.dist_matrix))
            cost = self.calculate_cost(individual)
            self.population.append((individual, cost))

    def calculate_cost(self, individual):
        cost = 0
        for i in range(1, len(individual)):
           cost += self.dist_matrix[individual[i - 1], individual[i]]
        # Adding the cost to return to the starting point to complete the cycle
        cost += self.dist_matrix[individual[-1], individual[0]]
        return cost

    def calculate_latency(self, individual):
        latency = 0
        total_latency = 0
        for i in range(1, len(individual)):
            latency += self.dist_matrix[individual[i - 1], individual[i]]
            total_latency += latency
        return total_latency

    def migrate(self, emigrate, immigrate):
        for i in range(len(immigrate)):
            if random.random() < self.mutation_prob:
                swap_idx = np.random.randint(0, len(emigrate))
                immigrate[i], immigrate[swap_idx] = immigrate[swap_idx], immigrate[i]
        return immigrate

    def optimize(self):
        start_time = time.time()
        self.initialize_population()
        self.population.sort(key=lambda x: x[1])

        for gen in range(self.max_gen):
            new_population = self.population[:self.elite_count]
            for i in range(self.elite_count, self.population_size):
                emigrate_idx = random.randint(0, self.elite_count - 1)
                immigrate_idx = random.randint(self.elite_count, self.population_size - 1)
                emigrate = self.population[emigrate_idx][0].copy()
                immigrate = self.population[immigrate_idx][0].copy()
                new_individual = self.migrate(emigrate, immigrate)
                new_cost = self.calculate_latency(new_individual)
                new_population.append((new_individual, new_cost))

            self.population = new_population
            self.population.sort(key=lambda x: x[1])

            if self.population[0][1] < self.best_cost:
                self.best_cost = self.population[0][1]
                self.best_solution = self.population[0][0]
            print(f"Generation {gen+1}: Best Cost = {self.best_cost}")

        end_time = time.time()
        avg_time = (end_time - start_time) / self.max_gen

        return self.best_solution, self.best_cost * 10, avg_time * 10

# Load the dataset
filename = r'/content/drive/MyDrive/TSPLIB95 folder/rat195.tsp'
nodes = load_tsplib95_instance(filename)
dist_matrix = calculate_distance_matrix(nodes)

# Run BBO algorithm
bbo = BBO(dist_matrix, population_size=300, max_gen=6, elite_count=20, mutation_prob=1)
best_solution, best_cost, avg_time = bbo.optimize()

print("Best solution found:", best_solution)
print("Cost of the best solution:", best_cost)
print("Average time per generation:", avg_time)

Generation 1: Best Cost = 20889.017781840408
Generation 2: Best Cost = 20889.017781840408
Generation 3: Best Cost = 20889.017781840408
Generation 4: Best Cost = 20889.017781840408
Generation 5: Best Cost = 20889.017781840408
Generation 6: Best Cost = 20889.017781840408
Best solution found: [184  13  19  21  84 112 172  93 107 179  73   3 190 173 191 176  27  87
   2 126 134 109 110 153  44 127 189  48 185 163 103  50  74  55  37  53
  20 138 122   7  58   9  80 130  65  39  12  62  68  15 119 180 187  25
 140  26 182  45  52  99  42 169 142  81  95 151  23  98 192 158  96 147
   4  46  60 157 101 129  77  66 128   1  36  89  88  57  49  72  86  59
  14  54  85  47  61 111   0  24  38 137  67  40  29 188  51  71   5 148
  28   8 154  79 183 143  17  92  18 145 155  63 100  35 165  83 136 166
 167 104 120  70 113 144  78  75  34  22 121 170 161 168  10 102 178 156
 133 132 125 159  11 175 115 193 116 177 106 118 162 114 194 152  30  33
  82  64  43  69 108 171 181 150  76  97  94 174 160

In [ ]:
# Run 9: population_size=600, max_gen=6, elite_count=20, mutation_prob=0.01
import numpy as np
import tsplib95
import random
import math
import time
from collections import namedtuple

# Load TSPLIB95 dataset
def load_tsplib95_instance(filename):
    problem = tsplib95.load(filename)
    if problem.is_depictable():
        nodes = np.array([problem.get_display(node) for node in problem.get_nodes()])
    else:
        print("This problem instance does not have display data.")
        nodes = None
    return nodes

# Calculate the distance matrix
def calculate_distance_matrix(nodes):
    n = len(nodes)
    dist_matrix = np.zeros((n, n))
    for i in range(n):
        for j in range(n):
            dist_matrix[i, j] = np.linalg.norm(nodes[i] - nodes[j])
    return dist_matrix

# Define the BBO algorithm
class BBO:
    def __init__(self, dist_matrix, population_size=600, max_gen=6, elite_count=20, mutation_prob=0.01):
        self.dist_matrix = dist_matrix
        self.population_size = population_size
        self.max_gen = max_gen
        self.elite_count = elite_count
        self.mutation_prob = mutation_prob
        self.population = []
        self.elite = []
        self.best_solution = None
        self.best_cost = float('inf')

    def initialize_population(self):
        for _ in range(self.population_size):
            individual = np.random.permutation(len(self.dist_matrix))
            cost = self.calculate_cost(individual)
            self.population.append((individual, cost))

    def calculate_cost(self, individual):
        cost = 0
        for i in range(1, len(individual)):
           cost += self.dist_matrix[individual[i - 1], individual[i]]
        # Adding the cost to return to the starting point to complete the cycle
        cost += self.dist_matrix[individual[-1], individual[0]]
        return cost

    def calculate_latency(self, individual):
        latency = 0
        total_latency = 0
        for i in range(1, len(individual)):
            latency += self.dist_matrix[individual[i - 1], individual[i]]
            total_latency += latency
        return total_latency

    def migrate(self, emigrate, immigrate):
        for i in range(len(immigrate)):
            if random.random() < self.mutation_prob:
                swap_idx = np.random.randint(0, len(emigrate))
                immigrate[i], immigrate[swap_idx] = immigrate[swap_idx], immigrate[i]
        return immigrate

    def optimize(self):
        start_time = time.time()
        self.initialize_population()
        self.population.sort(key=lambda x: x[1])

        for gen in range(self.max_gen):
            new_population = self.population[:self.elite_count]
            for i in range(self.elite_count, self.population_size):
                emigrate_idx = random.randint(0, self.elite_count - 1)
                immigrate_idx = random.randint(self.elite_count, self.population_size - 1)
                emigrate = self.population[emigrate_idx][0].copy()
                immigrate = self.population[immigrate_idx][0].copy()
                new_individual = self.migrate(emigrate, immigrate)
                new_cost = self.calculate_latency(new_individual)
                new_population.append((new_individual, new_cost))

            self.population = new_population
            self.population.sort(key=lambda x: x[1])

            if self.population[0][1] < self.best_cost:
                self.best_cost = self.population[0][1]
                self.best_solution = self.population[0][0]
            print(f"Generation {gen+1}: Best Cost = {self.best_cost}")

        end_time = time.time()
        avg_time = (end_time - start_time) / self.max_gen

        return self.best_solution, self.best_cost * 10, avg_time * 10

# Load the dataset
filename = r'/content/drive/MyDrive/TSPLIB95 folder/rat195.tsp'
nodes = load_tsplib95_instance(filename)
dist_matrix = calculate_distance_matrix(nodes)

# Run BBO algorithm
bbo = BBO(dist_matrix, population_size=600, max_gen=6, elite_count=20, mutation_prob=0.01)
best_solution, best_cost, avg_time = bbo.optimize()

print("Best solution found:", best_solution)
print("Cost of the best solution:", best_cost)
print("Average time per generation:", avg_time)

Generation 1: Best Cost = 19997.818765018787
Generation 2: Best Cost = 19997.818765018787
Generation 3: Best Cost = 19997.818765018787
Generation 4: Best Cost = 19997.818765018787
Generation 5: Best Cost = 19997.818765018787
Generation 6: Best Cost = 19997.818765018787
Best solution found: [ 97  53  52  57 104 171 167 190  70  84  55  61  12  85  47  91 111  68
  44  31 147 112  77 113  72 125 178 148 145  83 151 138 144  93  80 177
  24 158 101 156 170 140  16  25  27  14 136 175 114   6  17  89  81  99
 116  15  39  45 109  54 117 169 103  11  75  86 159 120 187  51  10  59
  33  18  73 181 118   7 179   5  79 168 153  58 107  20  50 162  65 110
  78 133  42 194 106  88  98 105  30 180  36   1  19 135 139  38  96  82
  74 134 127 164 115 128  41  23  63 122 185  26 146 176  56  62  71  34
   3  67  60 154  49 189 193 137 141 172  87   8  21  32  64  48 124 123
 130 152 155   4  40 183 108  92 142  22 149 150   0  37 182  66 126 102
  29 157 119 132 100 166 121 191 165 174  94 163 184

In [ ]:
# Run 10: population_size=700, max_gen=6, elite_count=500, mutation_prob=0.01
import numpy as np
import tsplib95
import random
import math
import time
from collections import namedtuple

# Load TSPLIB95 dataset
def load_tsplib95_instance(filename):
    problem = tsplib95.load(filename)
    if problem.is_depictable():
        nodes = np.array([problem.get_display(node) for node in problem.get_nodes()])
    else:
        print("This problem instance does not have display data.")
        nodes = None
    return nodes

# Calculate the distance matrix
def calculate_distance_matrix(nodes):
    n = len(nodes)
    dist_matrix = np.zeros((n, n))
    for i in range(n):
        for j in range(n):
            dist_matrix[i, j] = np.linalg.norm(nodes[i] - nodes[j])
    return dist_matrix

# Define the BBO algorithm
class BBO:
    def __init__(self, dist_matrix, population_size=700, max_gen=6, elite_count=500, mutation_prob=0.01):
        self.dist_matrix = dist_matrix
        self.population_size = population_size
        self.max_gen = max_gen
        self.elite_count = elite_count
        self.mutation_prob = mutation_prob
        self.population = []
        self.elite = []
        self.best_solution = None
        self.best_cost = float('inf')

    def initialize_population(self):
        for _ in range(self.population_size):
            individual = np.random.permutation(len(self.dist_matrix))
            cost = self.calculate_cost(individual)
            self.population.append((individual, cost))

    def calculate_cost(self, individual):
        cost = 0
        for i in range(1, len(individual)):
           cost += self.dist_matrix[individual[i - 1], individual[i]]
        # Adding the cost to return to the starting point to complete the cycle
        cost += self.dist_matrix[individual[-1], individual[0]]
        return cost

    def calculate_latency(self, individual):
        latency = 0
        total_latency = 0
        for i in range(1, len(individual)):
            latency += self.dist_matrix[individual[i - 1], individual[i]]
            total_latency += latency
        return total_latency

    def migrate(self, emigrate, immigrate):
        for i in range(len(immigrate)):
            if random.random() < self.mutation_prob:
                swap_idx = np.random.randint(0, len(emigrate))
                immigrate[i], immigrate[swap_idx] = immigrate[swap_idx], immigrate[i]
        return immigrate

    def optimize(self):
        start_time = time.time()
        self.initialize_population()
        self.population.sort(key=lambda x: x[1])

        for gen in range(self.max_gen):
            new_population = self.population[:self.elite_count]
            for i in range(self.elite_count, self.population_size):
                emigrate_idx = random.randint(0, self.elite_count - 1)
                immigrate_idx = random.randint(self.elite_count, self.population_size - 1)
                emigrate = self.population[emigrate_idx][0].copy()
                immigrate = self.population[immigrate_idx][0].copy()
                new_individual = self.migrate(emigrate, immigrate)
                new_cost = self.calculate_latency(new_individual)
                new_population.append((new_individual, new_cost))

            self.population = new_population
            self.population.sort(key=lambda x: x[1])

            if self.population[0][1] < self.best_cost:
                self.best_cost = self.population[0][1]
                self.best_solution = self.population[0][0]
            print(f"Generation {gen+1}: Best Cost = {self.best_cost}")

        end_time = time.time()
        avg_time = (end_time - start_time) / self.max_gen

        return self.best_solution, self.best_cost * 10, avg_time * 10

# Load the dataset
filename = r'/content/drive/MyDrive/TSPLIB95 folder/rat195.tsp'
nodes = load_tsplib95_instance(filename)
dist_matrix = calculate_distance_matrix(nodes)

# Run BBO algorithm
bbo = BBO(dist_matrix, population_size=700, max_gen=6, elite_count=500, mutation_prob=0.01)
best_solution, best_cost, avg_time = bbo.optimize()

print("Best solution found:", best_solution)
print("Cost of the best solution:", best_cost)
print("Average time per generation:", avg_time)

Generation 1: Best Cost = 20107.91344053085
Generation 2: Best Cost = 20107.91344053085
Generation 3: Best Cost = 20107.91344053085
Generation 4: Best Cost = 20107.91344053085
Generation 5: Best Cost = 20107.91344053085
Generation 6: Best Cost = 20107.91344053085
Best solution found: [ 51  90 171 110  94 168 184   1 102 159  78 130 141 165  13 123  47  33
 100  50  34  95  61 117 180 166  16  52   3 191 169 127  72  69 142 174
 160  17  57  64   7   2 107   8  81  15  79  89 176 155 187 177 136 150
  54  18 135 148 186 147 194 162  85  77  22 126   9 124   6 131 112  82
 120  19 129 163 156  49  30   4 139 145 149  25  28  76  48 137 140 109
 128 189 153   5 164 115 178  37  23  14  62  10  96 179  32  97  92  71
 181 103 146 101  43 134 104  60  39  74  36 161  75  58  20  59 119  83
 151 193 170  88  38  35  63  27  65 108  56  91  42  21 190  67  66  87
  55 157  70  41  98  40  31  44  45  26  73 158 172 167  24 183 185 116
 132 106  11  46 125  93 114 138 105 121 118 111 175 113  